<div style="background-image:url(images/meschede-seismic-waves.png); padding: 10px 30px 20px 30px; background-size:cover; background-opacity:50%; border-radius:5px; background-position: 0px -200px">
<p style="float:right; margin-top:20px; padding: 20px 60px 0px 10px; background:rgba(255,255,255,0.75); border-radius:10px;">
<img width="400px" src=images/obspy_logo_full_524x179px.png?raw=true>
</p>

<h1 style="color:#BBB; padding-bottom: 80px">MESS 2016 - Practicals</h1>

<h2 style="color:#FFF; padding-bottom: 30px">Master Event Relocation</h2>

</div>

### Please execute first cell to have plots show up inline (and scrollable/zoomable)
### If you are unclear about how to use some ObsPy function: use search box at http://docs.obspy.org

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend("nbagg")
plt.style.use("bmh")
import warnings
warnings.filterwarnings("ignore", message='The resource identifier.*')

 * load clustering catalog using obspy (file `data/clustering45_events.quakeml`, `read_events` function)
 * load station inventory (file `data/station_UH_all.stationxml`, `read_inventory` function)
 * load clustering similarity matrix using numpy.load (file `data/clustering45_similarity.npy`, numpy `load` function)

In [ ]:
# TODO: 

# we use these variable names below, so please adhere to these names
cat = 
similarity = 
inv = 

 * execute cell below to get a display of the similarity and dendrogram again

In [ ]:
# Plot read in similarity and visualize dendrogram
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import squareform
import matplotlib.pyplot as plt
from obspy.imaging.cm import viridis


def plot_similarity(similarity, dissimilarity_threshold=0.1, method="complete"):
    """
    Plot similarity matrix
    
    similarity: numpy ndarray
    dissimilarity_threshold: float, cut-off value for clustering
    method: str, agglomerative clustering criterion
    """
    fig = plt.figure(figsize=(10, 15))
    ax1 = fig.add_subplot(2, 1, 1)
    ax2 = fig.add_subplot(2, 1, 2)

    im = ax1.imshow(similarity, interpolation="nearest", cmap=viridis)
    ax1.set_xlabel("event index")
    ax1.set_ylabel("event index")
    ax1.grid(False)
    cb = plt.colorbar(mappable=im, ax=ax1)
    cb.ax.set_ylabel("similarity")

    distance = 1 - similarity
    link = linkage(squareform(distance), method=method)

    dendrogram(link, color_threshold=dissimilarity_threshold, orientation="right", ax=ax2)
    
    clusters = fcluster(link, dissimilarity_threshold, criterion="distance")
    for i in range(1, clusters.max() + 1):
        indices = np.where(clusters == i)[0].tolist()
        print("cluster no. {}: {}".format(i, indices)) 
    
    ax2.axvline(dissimilarity_threshold, color="k", ls="--")
    ax2.set_xlabel("dissimilarity")
    plt.tight_layout()
    plt.show()
    

plot_similarity(similarity, dissimilarity_threshold=0.1)

![Master event relocation](images/master-event.png)

 * take the first station from the inventory and the first event in the catalog and set up a unit vector pointing from source to station (strong simplification for the sake of simplicity, assuming a simple straight line for the ray)
   * use e.g. `gps2dist_azimuth` function from `obspy.geodetics` and `math` module (`sin`, `cos`, `pi`, `radians`, ...)
   * note: convert from azimuth to mathematical angle..
 * you can use the plotting function in the box below to check your calculation
 * (Gold card members: use take-off angle and azimuth information for a phase stored as an arrival in origin)

In [ ]:
# TODO:


# we use these names as the three components of the unit vector,
# please stick to these variable names so the box below can plot your vector
dx =
dy =
dz =

In [ ]:
from matplotlib.ticker import FormatStrFormatter

def plot_unit_vector(event, station, dx, dy, dz, fig=None, show=True):
    """
    Plot event, station and unit vector from source to station
    
    event: obspy Event object
    station: obspy Station object
    dx, dy, dz: floats
    """
    origin = event.origins[0]
    if not fig:
        fig = plt.figure()
    ax = fig.add_subplot(111)
    # plot markers for event + station
    ax.scatter(origin.longitude, origin.latitude, s=300, marker="*", color="g")
    ax.scatter(station.longitude, station.latitude, s=200, marker="v")
    # add text labels for event + station
    ax.text(origin.longitude, origin.latitude, "  " + str(event.resource_id).split("/")[-1])
    ax.text(station.longitude, station.latitude, "  " + station.code)
    # plot unit vector (scale it arbitrarily to the plot size)
    scale_factor = 0.03
    # need to take latitude into account: no equal data scale in meters!
    latitude_correction = cos(radians(origin.latitude))
    dx *= scale_factor / latitude_correction
    dy *= scale_factor
    ax.plot([origin.longitude, origin.longitude + dx],
            [origin.latitude, origin.latitude + dy],
            "r-", zorder=-1)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.xaxis.set_major_formatter(FormatStrFormatter("%s"))
    ax.yaxis.set_major_formatter(FormatStrFormatter("%s"))
    ax.set_aspect(latitude_correction, adjustable="datalim")
    if show:
        plt.show()


plot_unit_vector(event, station, dx, dy, dz)

 * below are two functions that do the unit vector calculation for all stations in the inventory
 * execute the box, so the functions are available for the master-event relocation below
 * (Gold card member exercise: replace unit vector calculation with take-off angle and azimuth from arrival information stored in origin, coming from NonLinLoc location run)

In [ ]:
from math import sin, cos, radians
from obspy.geodetics import gps2dist_azimuth


def coordinates_to_unitvector(event, station_code, inventory, plot=False, fig=None, show=True):
    """
    Compute (x, y, z) unit vector pointing from source to receiver along the
    straight line connecting origin and receiver.
    
    event: obspy Event object
    station_code: station name as string (e.g. "UH1")
    inventory: obspy Inventory object
    plot: boolean, wheteher to show a plot
    
    returns: unit vector as three floats (x, y, z)
    """
    origin = event.origins[0]
    station_coordinates = inventory.get_coordinates(
        "BW.{}..EHZ".format(station_code), datetime=origin.time)
    distance, azimuth, backazimuth = gps2dist_azimuth(
        origin.latitude, origin.longitude,
        station_coordinates['latitude'], station_coordinates['longitude']) 
    dz = origin.depth + station_coordinates['elevation']
    angle = radians(90 - azimuth)
    dx = distance * cos(angle)
    dy = distance * sin(angle)
    norm = np.linalg.norm((dx, dy, dz))
    dx /=  norm
    dy /=  norm
    dz /=  norm

    if plot:
        sta=None
        for net in inv:
            for sta in net:
                if sta.code == station_code:
                    break
        plot_unit_vector(event, sta, dx, dy, dz, fig=fig, show=show)

    return (dx, dy, dz)

def all_coordinates_to_unitvectors(event, inventory, plot=True):
    unit_vectors = {}
    station_codes = set([sta.code for net in inv for sta in net])
    if plot:
        fig = plt.figure(figsize=(11, 4))
    else:
        fig=None
    for sta in station_codes:
        unit_vectors[sta] = coordinates_to_unitvector(
            cat[0], sta, inv, plot=plot, fig=fig, show=False)
    if plot:
        plt.show()
    return unit_vectors

event = cat[0]
all_coordinates_to_unitvectors(event, inv)

 * below are two helper functions..
   * ..to get the matching pick from a second event
   * ..to calculate a differential travel time for a given pick from a master event and a slave event
 * execute the box, so the functions are available for the master-event relocation below

In [ ]:
from obspy import read
from obspy.signal.cross_correlation import xcorr_pick_correction

ST = read("data/clustering45_waveforms.mseed")


def get_corresponding_pick(pick, event):
    """
    Extract the corresponding pick from an event (slave event),
    given a pick from a different event (master event).
    
    pick: obspy Pick object
    event: obspy Event object
    
    returns: Pick object or None if no corresponding pick was found
    """
    pick2 = None
    for pick2 in event.picks:
        if (pick2.waveform_id.station_code == pick.waveform_id.station_code
                and pick2.phase_hint == pick.phase_hint):
            return pick2
    else:
        return None
    

def get_differential_traveltime(pick_master, event_slave, st=None, refine=False,
                                plot=False, xcorr_threshold=0.7):
    """
    Get differential travel time given a pick from a master event and a slave event.
    Optionally refine differential travel time with cross correlation.
    
    pick_master: obspy Pick object
    event_slave: obspy Event object
    st: obspy Stream object containing waveforms for cross correlation
    refine: boolean, whether to cross-correlation-align picks
    """
    # get matching 
    pick_slave = get_corresponding_pick(pick_master, event_slave)
    if not pick_slave or not pick_slave.time:
        return None
    dt = pick_slave.time - pick_master.time
    # refine differential travel time by cross-correlation-aligning of picks
    if refine:
        tr_master = [tr_ for tr_ in ST
                     if tr_.stats.starttime < pick_master.time < tr_.stats.endtime
                     and tr_.id == pick_master.waveform_id.get_seed_string()][0].copy()
        tr_slave = [tr_ for tr_ in ST
                    if tr_.stats.starttime < pick_slave.time < tr_.stats.endtime
                    and tr_.id == pick_slave.waveform_id.get_seed_string()][0].copy()
        # cross-correlation pick alignment
        t_correction, coeff = xcorr_pick_correction(
            pick_master.time, tr_master, pick_slave.time, tr_slave,
            t_before=0.05, t_after=0.2, cc_maxlag=0.1, filter="bandpass",
            filter_options={"freqmin": 1, "freqmax": 20}, plot=plot)
        # discard picks that have low cross-correlation
        if coeff < xcorr_threshold:
            print("Pick discarded (coeff: {:.2f})".format(coeff))
            return None
        # adjust slave pick time
        dt += t_correction
    return dt

 * below are two functions to execute the master-event relocation, execute the code cells to be able to use them below

![Master event relocation](images/master-event.png)
![Master event relocation](images/pinv.png)

 * (Gold card member exercise: Add weighting (e.g. by correlation coefficient of pick) to the inversion)

In [ ]:
from obspy import Catalog, read

def relocate_event(master, slave, inventory, velocities, refine):
    """
    Relocate a single slave event relative to master event.
    
    master: obspy Event object
    slave: obspy Event object
    inventory: obspy Inventory object
    velocities: dictionary with keys "P" and "S" giving wave velocities in source region in m/s.
    refine: boolean, whether to cross-correlation-align picks
    
    returns: Hypocenter of slave location in relative coordinates to master (dt, dx, dy, dz)
    """
    unit_vectors = all_coordinates_to_unitvectors(master, inv, plot=False)
    data_vector = []
    forward_operator_matrix = []
    # assemble data vector and forward operator matrix,
    # iterating over all usable picks
    for pick in master.picks:
        if pick.waveform_id.station_code not in unit_vectors:
            continue
        # get differential travel time
        dt = get_differential_traveltime(pick, slave, refine=refine)
        if dt is None:
            continue
        # extract the correct unit vector for the pick
        x, y, z = unit_vectors[pick.waveform_id.station_code]
        vel = velocities[pick.phase_hint]
        # forward operator setup, see image above
        operator_line = (1.0, -x/vel, -y/vel, -z/vel)
        data_vector.append(dt)
        forward_operator_matrix.append(operator_line)
    # convert lists to numpy arrays
    data_vector = np.array(data_vector)
    forward_operator_matrix = np.array(forward_operator_matrix)
    # compute moore-penrose pseudo inverse
    inverse_operator_matrix = np.linalg.pinv(forward_operator_matrix)
    # apply inverted operator to data vector
    relative_location = np.dot(inverse_operator_matrix, data_vector)
    dt_origin, dx, dy, dz = relative_location
    return dt_origin, dx, dy, dz
    

def relocate_events(catalog, master_event_id, inventory, velocities, refine):
    """
    Relocate a set of events, specifying the index which event should be used as master event.
    Shows a plot of the relocation results with master event at coordinate origin.
    
    catalog: obspy Catalog object
    master_event_id: integer, master event number (as shown in similarity plot above)
    inventory: obspy Inventory object
    velocities: dictionary with keys "P" and "S" giving wave velocities in source region in m/s.
    refine: boolean, whether to cross-correlation-align picks
    
    returns: Hypocenter of slave location in relative coordinates to master (dt, dx, dy, dz)
    """
    for event in catalog:
        if int(str(event.resource_id).split("/")[-1]) == master_event_id:
            master = event
            break
    else:
        raise Exception("Could not find master event with ID {}".format(master_event_id))
    slaves = Catalog()
    for event in catalog:
        if event != master:
            slaves.append(event)

    master_event_number = str(master.resource_id).split("/")[-1]
    plt.figure()
    plt.title("Master Event: {}".format(master_event_number))
    plt.scatter(0, 0, s=200, marker="*", color="r")
    plt.text(0, 0, "  " + master_event_number)
    for slave in slaves:
        dt_origin, dx, dy, dz = relocate_event(
            master, slave, inventory, velocities, refine=refine)
        # print dt_origin, dx, dy, dz
        plt.scatter(dx, dy, s=200, marker="*", color="g")
        plt.text(dx, dy, "  " + str(slave.resource_id).split("/")[-1])
    plt.gca().set_aspect("equal", adjustable="datalim")
    plt.xlabel("x [m]")
    plt.ylabel("y [m]")
    plt.show()

 * set up a new catalog with one of the clusters shown in the dendrogram above (use the event indices shown in the dendrogram)
 * set up a dictionary with keys "P" and "S" (for velocities), use P-wave velocity 5000 (m/s) and S-wave velocity 2700 (m/s)

In [ ]:
# TODO:
from obspy import Catalog, read

new_cat = 

velocities =

 * visualize the master-event relocations using 2-3 events as a master in turn,
   without cross-correlation refinement of picks (`refine=False`); use `relocate_events()` function defined in the above box
 * what differences and/or common aspects do you see?

In [ ]:
# TODO:
relocate_events(...)
relocate_events(...)
relocate_events(...)

 * again: visualize the master-event relocations using 2-3 events as a master in turn but now *with* cross-correlation pick refinement (option `refine=True`)
 * what are the differences to before (event separation distances, general looks of the cluster, etc.)?
 * compare relative location of events to their positioning in the dendrogram above
 * (Gold card members: add the relocation results to the existing event object as new origin objects (use `obspy.signal.util.util_lon_lat` to convert relative to geographical coordinates), set them as preferred origin, save to QuakeML)

In [ ]:
# TODO:
import warnings
# cross correlation pick alignment shows many warnings on bad correlations,
# here we catch most of them to have less verbose output
with warnings.catch_warnings():
    warnings.simplefilter("once")
    relocate_events(...)
    relocate_events(...)
    relocate_events(...)